## Packages Installation

First, install the `holisticai` package if you haven't already:
```bash
!pip install holisticai[all]
```
Then, import the necessary libraries.

# Setup Logging

In [6]:
import logging
import sys
import warnings

warnings.filterwarnings("ignore")

logger = logging.getLogger()
logger.setLevel(logging.INFO)
console_handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

In [7]:
import pandas as pd
from holisticai.bias.metrics import classification_bias_metrics
from holisticai.datasets import load_dataset
from holisticai.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

## Dataset loading

In [8]:
dataset = load_dataset('adult', protected_attribute='sex')
train_test = dataset.train_test_split(test_size=0.2, random_state=42)

train = train_test['train']
test = train_test['test']

dataset

Dataset({
        features: ['X', 'y', 'group_a', 'group_b'],
        num_rows: 45222
    })

In [9]:
# Define inprocessing model
model = LogisticRegression()

# Standardize data and fit model
scaler = StandardScaler()
X_train = scaler.fit_transform(train['X'])
model.fit(train['X'], train['y'])

# Standardize data and predict
X_test = scaler.transform(test['X'])
y_pred = model.predict(X_test)

# Evaluate bias metrics
metrics = classification_bias_metrics(test['group_a'], test['group_b'], y_pred, test['y'], metric_type='both')
metrics

,Value,Reference
Metric,,
Statistical Parity,0.394067,0
Disparate Impact,2.958943,1
Four Fifths Rule,0.337959,1
Cohen D,0.849856,0
2SD Rule,35.128112,0
Equality of Opportunity Difference,0.237860,0
False Positive Rate Difference,0.333779,0
Average Odds Difference,0.285820,0
Accuracy Difference,-0.204467,0


Now that we have a clean dataset we can start defining the training and testing sets.

# 1. Adversarial Debiasing

### Traditional Implementation

In [10]:
# Define inprocessing model
from holisticai.bias.mitigation import AdversarialDebiasing

mitigator = AdversarialDebiasing(features_dim=train['X'].shape[1], batch_size=512, hidden_size=64, 
                                 adversary_loss_weight=3, verbose=1, use_debias=True, seed=42).transform_estimator()

# Standardize data and fit model
scaler = StandardScaler()
X_train = scaler.fit_transform(train['X'])
mitigator.fit(X_train, train['y'], train['group_a'], train['group_b'])
X_test = scaler.transform(test['X'])
y_pred = mitigator.predict(X_test)

# Evaluate bias metrics
metrics = classification_bias_metrics(test['group_a'], test['group_b'], y_pred, test['y'], metric_type='both')
metrics

2024-07-10 19:10:55,674 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - Step 1/710: Classifier Loss = 0.639912, Adversarial Loss = 0.765241
2024-07-10 19:10:55,674 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - Step 1/710: Classifier Loss = 0.639912, Adversarial Loss = 0.765241
2024-07-10 19:11:03,982 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - Step 101/710: Classifier Loss = 0.383029, Adversarial Loss = 0.618528
2024-07-10 19:11:03,982 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - Step 101/710: Classifier Loss = 0.383029, Adversarial Loss = 0.618528
2024-07-10 19:11:12,019 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - Step 201/710: Classifier Loss = 0.370479, Adversarial Loss = 0.620525
2024-07-10 19:11:12,019 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - 

,Value,Reference
Metric,,
Statistical Parity,0.020145,0
Disparate Impact,1.147575,1
Four Fifths Rule,0.871402,1
Cohen D,0.056414,0
2SD Rule,2.508389,0
Equality of Opportunity Difference,-0.266421,0
False Positive Rate Difference,-0.027236,0
Average Odds Difference,-0.146828,0
Accuracy Difference,-0.108707,0


### Pipeline Implementation

In [11]:
mitigator = AdversarialDebiasing(features_dim=train['X'].shape[1], batch_size=512, hidden_size=64, 
                                 adversary_loss_weight=3, verbose=1, use_debias=True, seed=42).transform_estimator()

# set up the pipeline
pipeline = Pipeline(steps=[('scalar', StandardScaler()), ('bm_inprocessing', mitigator)])
pipeline.fit(train['X'], train['y'], bm__group_a=train['group_a'], bm__group_b=train['group_b'])

# make predictions
y_pred_pipeline = pipeline.predict(test['X'], bm__group_a=test['group_a'], bm__group_b=test['group_b'])

# compute bias metrics
metrics_pipeline = classification_bias_metrics(test['group_a'], test['group_b'], y_pred_pipeline, test['y'], metric_type='both')
metrics_pipeline

2024-07-10 19:11:54,772 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - Step 1/710: Classifier Loss = 0.639912, Adversarial Loss = 0.765241
2024-07-10 19:11:54,772 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - Step 1/710: Classifier Loss = 0.639912, Adversarial Loss = 0.765241
2024-07-10 19:12:02,797 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - Step 101/710: Classifier Loss = 0.383029, Adversarial Loss = 0.618528
2024-07-10 19:12:02,797 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - Step 101/710: Classifier Loss = 0.383029, Adversarial Loss = 0.618528
2024-07-10 19:12:10,167 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - Step 201/710: Classifier Loss = 0.370479, Adversarial Loss = 0.620525
2024-07-10 19:12:10,167 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - 

,Value,Reference
Metric,,
Statistical Parity,0.020145,0
Disparate Impact,1.147575,1
Four Fifths Rule,0.871402,1
Cohen D,0.056414,0
2SD Rule,2.508389,0
Equality of Opportunity Difference,-0.266421,0
False Positive Rate Difference,-0.027236,0
Average Odds Difference,-0.146828,0
Accuracy Difference,-0.108707,0


### Comparison

In [12]:
pd.concat([metrics['Value'], metrics_pipeline], axis=1, keys=['Traditional', 'Pipeline'])

Traditional  Pipeline          
                                         Value     Value Reference
Metric                                                            
Statistical Parity                    0.020145  0.020145         0
Disparate Impact                      1.147575  1.147575         1
Four Fifths Rule                      0.871402  0.871402         1
Cohen D                               0.056414  0.056414         0
2SD Rule                              2.508389  2.508389         0
Equality of Opportunity Difference   -0.266421 -0.266421         0
False Positive Rate Difference       -0.027236 -0.027236         0
Average Odds Difference              -0.146828 -0.146828         0
Accuracy Difference                  -0.108707 -0.108707         0

# 2. Exponentiated Gradient

### Traditional Implementation

In [13]:
# Define inprocessing model
from holisticai.bias.mitigation import ExponentiatedGradientReduction

model = LogisticRegression()
mitigator = ExponentiatedGradientReduction(constraints="DemographicParity").transform_estimator(model)

# Standardize data and fit model
scaler = StandardScaler()
X_train = scaler.fit_transform(train['X'])
mitigator.fit(X_train, train['y'], train['group_a'], train['group_b'])

# Standardize data and predict
X_test = scaler.transform(test['X'])
y_pred = mitigator.predict(X_test)

# Evaluate bias metrics
metrics = classification_bias_metrics(test['group_a'], test['group_b'], y_pred, test['y'], metric_type='both')
metrics

,Value,Reference
Metric,,
Statistical Parity,-0.004007,0
Disparate Impact,0.974595,1
Four Fifths Rule,0.974595,1
Cohen D,-0.011071,0
2SD Rule,-0.492437,0
Equality of Opportunity Difference,-0.317883,0
False Positive Rate Difference,-0.046793,0
Average Odds Difference,-0.182338,0
Accuracy Difference,-0.098740,0


### Pipeline Implementation

In [14]:
mitigator = ExponentiatedGradientReduction(constraints="DemographicParity").transform_estimator(model)

# set up the pipeline
pipeline = Pipeline(steps=[('scalar', StandardScaler()), ('bm_inprocessing', mitigator)])
pipeline.fit(train['X'], train['y'], bm__group_a=train['group_a'], bm__group_b=train['group_b'])

# make predictions
y_pred_pipeline = pipeline.predict(test['X'], bm__group_a=test['group_a'], bm__group_b=test['group_b'])

# compute bias metrics
metrics_pipeline = classification_bias_metrics(test['group_a'], test['group_b'], y_pred_pipeline, test['y'], metric_type='both')
metrics_pipeline

,Value,Reference
Metric,,
Statistical Parity,-0.004007,0
Disparate Impact,0.974595,1
Four Fifths Rule,0.974595,1
Cohen D,-0.011071,0
2SD Rule,-0.492437,0
Equality of Opportunity Difference,-0.317883,0
False Positive Rate Difference,-0.046793,0
Average Odds Difference,-0.182338,0
Accuracy Difference,-0.098740,0


### Comparison

In [15]:
pd.concat([metrics['Value'], metrics_pipeline], axis=1, keys=['Traditional', 'Pipeline'])

Traditional  Pipeline          
                                         Value     Value Reference
Metric                                                            
Statistical Parity                   -0.004007 -0.004007         0
Disparate Impact                      0.974595  0.974595         1
Four Fifths Rule                      0.974595  0.974595         1
Cohen D                              -0.011071 -0.011071         0
2SD Rule                             -0.492437 -0.492437         0
Equality of Opportunity Difference   -0.317883 -0.317883         0
False Positive Rate Difference       -0.046793 -0.046793         0
Average Odds Difference              -0.182338 -0.182338         0
Accuracy Difference                  -0.098740 -0.098740         0

# 3. Grid Search Reduction

### Traditional Implementation

In [16]:
# Define inprocessing model
from holisticai.bias.mitigation import GridSearchReduction

model = LogisticRegression()
mitigator = GridSearchReduction(constraints="DemographicParity", loss='Square', min_val=-0.1, max_val=1.3, 
                                         grid_size=20).transform_estimator(model)

# Standardize data and fit model
scaler = StandardScaler()
X_train = scaler.fit_transform(train['X'])
mitigator.fit(X_train, train['y'], train['group_a'], train['group_b'])
X_test = scaler.transform(test['X'])
y_pred = mitigator.predict(X_test)

# Evaluate bias metrics
metrics = classification_bias_metrics(test['group_a'], test['group_b'], y_pred, test['y'], metric_type='both')
metrics

,Value,Reference
Metric,,
Statistical Parity,0.011524,0
Disparate Impact,1.076728,1
Four Fifths Rule,0.928739,1
Cohen D,0.031599,0
2SD Rule,1.405328,0
Equality of Opportunity Difference,-0.289828,0
False Positive Rate Difference,-0.035375,0
Average Odds Difference,-0.162602,0
Accuracy Difference,-0.101732,0


### Pipeline Implementation

In [17]:
mitigator = GridSearchReduction(constraints="DemographicParity", loss='Square', min_val=-0.1, max_val=1.3, 
                                         grid_size=20).transform_estimator(model)
# set up the pipeline
pipeline = Pipeline(steps=[('scalar', StandardScaler()), ('bm_inprocessing', mitigator)])
pipeline.fit(train['X'], train['y'], bm__group_a=train['group_a'], bm__group_b=train['group_b'])

# make predictions
y_pred_pipeline = pipeline.predict(test['X'], bm__group_a=test['group_a'], bm__group_b=test['group_b'])

# compute bias metrics
metrics_pipeline = classification_bias_metrics(test['group_a'], test['group_b'], y_pred_pipeline, test['y'], metric_type='both')
metrics_pipeline

,Value,Reference
Metric,,
Statistical Parity,0.011524,0
Disparate Impact,1.076728,1
Four Fifths Rule,0.928739,1
Cohen D,0.031599,0
2SD Rule,1.405328,0
Equality of Opportunity Difference,-0.289828,0
False Positive Rate Difference,-0.035375,0
Average Odds Difference,-0.162602,0
Accuracy Difference,-0.101732,0


### Comparison

In [18]:
pd.concat([metrics['Value'], metrics_pipeline], axis=1, keys=['Traditional', 'Pipeline'])

Traditional  Pipeline          
                                         Value     Value Reference
Metric                                                            
Statistical Parity                    0.011524  0.011524         0
Disparate Impact                      1.076728  1.076728         1
Four Fifths Rule                      0.928739  0.928739         1
Cohen D                               0.031599  0.031599         0
2SD Rule                              1.405328  1.405328         0
Equality of Opportunity Difference   -0.289828 -0.289828         0
False Positive Rate Difference       -0.035375 -0.035375         0
Average Odds Difference              -0.162602 -0.162602         0
Accuracy Difference                  -0.101732 -0.101732         0

# 4. Meta Fair Classifier

### Traditional Implementation

In [19]:
# Define inprocessing model
from holisticai.bias.mitigation import MetaFairClassifier

model = LogisticRegression()
mitigator = MetaFairClassifier(constraint="StatisticalRate", verbose=1, seed=42).transform_estimator(_)

# Standardize data and fit model
scaler = StandardScaler()
X_train = scaler.fit_transform(train['X'])
mitigator.fit(X_train, train['y'], train['group_a'], train['group_b'])

# Standardize data and predict
X_test = scaler.transform(test['X'])
y_pred = mitigator.predict(X_test)

# Evaluate bias metrics
metrics = classification_bias_metrics(test['group_a'], test['group_b'], y_pred, test['y'], metric_type='both')
metrics

[elapsed time: 00:00:04 | iter:8/8 | accuracy:0.6313 | gamma:0.6322]


,Value,Reference
Metric,,
Statistical Parity,0.085380,0
Disparate Impact,1.159979,1
Four Fifths Rule,0.862085,1
Cohen D,0.174269,0
2SD Rule,7.725735,0
Equality of Opportunity Difference,-0.002836,0
False Positive Rate Difference,-0.003188,0
Average Odds Difference,-0.003012,0
Accuracy Difference,0.082562,0


### Pipeline Implementation

In [20]:
mitigator = MetaFairClassifier(constraint="StatisticalRate", verbose=1, seed=42).transform_estimator(_)
# set up the pipeline
pipeline = Pipeline(steps=[('scalar', StandardScaler()), ('bm_inprocessing', mitigator)])
pipeline.fit(train['X'], train['y'], bm__group_a=train['group_a'], bm__group_b=train['group_b'])

# make predictions
y_pred_pipeline = pipeline.predict(test['X'], bm__group_a=test['group_a'], bm__group_b=test['group_b'])

# compute bias metrics
metrics_pipeline = classification_bias_metrics(test['group_a'], test['group_b'], y_pred_pipeline, test['y'], metric_type='both')
metrics_pipeline

[elapsed time: 00:00:04 | iter:8/8 | accuracy:0.6313 | gamma:0.6322]


,Value,Reference
Metric,,
Statistical Parity,0.085380,0
Disparate Impact,1.159979,1
Four Fifths Rule,0.862085,1
Cohen D,0.174269,0
2SD Rule,7.725735,0
Equality of Opportunity Difference,-0.002836,0
False Positive Rate Difference,-0.003188,0
Average Odds Difference,-0.003012,0
Accuracy Difference,0.082562,0


### Comparison

In [21]:
pd.concat([metrics['Value'], metrics_pipeline], axis=1, keys=['Traditional', 'Pipeline'])

Traditional  Pipeline          
                                         Value     Value Reference
Metric                                                            
Statistical Parity                    0.085380  0.085380         0
Disparate Impact                      1.159979  1.159979         1
Four Fifths Rule                      0.862085  0.862085         1
Cohen D                               0.174269  0.174269         0
2SD Rule                              7.725735  7.725735         0
Equality of Opportunity Difference   -0.002836 -0.002836         0
False Positive Rate Difference       -0.003188 -0.003188         0
Average Odds Difference              -0.003012 -0.003012         0
Accuracy Difference                   0.082562  0.082562         0

# 5. Prejudice Remover

### Traditional Implementation

In [22]:
# Define inprocessing model
from holisticai.bias.mitigation import PrejudiceRemover

model = LogisticRegression()
mitigator = PrejudiceRemover(maxiter=100, fit_intercept=True, verbose=1, print_interval=1).transform_estimator(model)

# Standardize data and fit model
scaler = StandardScaler()
X_train = scaler.fit_transform(train['X'])
mitigator.fit(X_train, train['y'], train['group_a'], train['group_b'])

# Standardize data and predict
X_test = scaler.transform(test['X'])
y_pred = mitigator.predict(X_test, test['group_a'], test['group_b'])

# Evaluate bias metrics
metrics = classification_bias_metrics(test['group_a'], test['group_b'], y_pred, test['y'], metric_type='both')
metrics

[elapsed time: 00:00:02 | iter:5/100 | loss:13103.7994]
[elapsed time: 00:00:08 | Best Loss : 13103.7994]


,Value,Reference
Metric,,
Statistical Parity,0.108639,0
Disparate Impact,2.114219,1
Four Fifths Rule,0.472988,1
Cohen D,0.291181,0
2SD Rule,12.833175,0
Equality of Opportunity Difference,-0.037908,0
False Positive Rate Difference,0.030534,0
Average Odds Difference,-0.003687,0
Accuracy Difference,-0.110517,0


### Pipeline Implementation

In [23]:
mitigator = PrejudiceRemover(maxiter=100, fit_intercept=True, verbose=1, print_interval=1).transform_estimator(model)

# set up the pipeline
pipeline = Pipeline(steps=[('scalar', StandardScaler()), ('bm_inprocessing', mitigator)])
pipeline.fit(train['X'], train['y'], bm__group_a=train['group_a'], bm__group_b=train['group_b'])

# make predictions
y_pred_pipeline = pipeline.predict(test['X'], bm__group_a=test['group_a'], bm__group_b=test['group_b'])

# compute bias metrics
metrics_pipeline = classification_bias_metrics(test['group_a'], test['group_b'], y_pred_pipeline, test['y'], metric_type='both')
metrics_pipeline

[elapsed time: 00:00:02 | iter:5/100 | loss:13103.7994]
[elapsed time: 00:00:08 | Best Loss : 13103.7994]


,Value,Reference
Metric,,
Statistical Parity,0.108639,0
Disparate Impact,2.114219,1
Four Fifths Rule,0.472988,1
Cohen D,0.291181,0
2SD Rule,12.833175,0
Equality of Opportunity Difference,-0.037908,0
False Positive Rate Difference,0.030534,0
Average Odds Difference,-0.003687,0
Accuracy Difference,-0.110517,0


### Comparison

In [24]:
pd.concat([metrics['Value'], metrics_pipeline], axis=1, keys=['Traditional', 'Pipeline'])

Traditional   Pipeline          
                                         Value      Value Reference
Metric                                                             
Statistical Parity                    0.108639   0.108639         0
Disparate Impact                      2.114219   2.114219         1
Four Fifths Rule                      0.472988   0.472988         1
Cohen D                               0.291181   0.291181         0
2SD Rule                             12.833175  12.833175         0
Equality of Opportunity Difference   -0.037908  -0.037908         0
False Positive Rate Difference        0.030534   0.030534         0
Average Odds Difference              -0.003687  -0.003687         0
Accuracy Difference                  -0.110517  -0.110517         0